In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 5.1MB/s 
     |████████████████████████████████| 552kB 37.8MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=deac5dc17dbb94048cc4d81408ba3a8e5ff8339a6eee06791e930c6059d0be40
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32042 sha256=ac073faa2df89018ce79cb59a3f53ad7b7b2034c35ce2a3c4ab3d3c6221dc07f
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis funcy


In [0]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_recall_fscore_support
from gensim.models.keyedvectors import KeyedVectors
from gensim.parsing.preprocessing import remove_stopwords, strip_numeric, preprocess_string
import numpy as np
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from gensim.matutils import jaccard_distance
import pickle
import pyLDAvis.gensim
from joblib import Parallel, delayed, cpu_count
from gensim.sklearn_api import LdaTransformer
from gensim.models import CoherenceModel, LdaModel
from gensim.corpora.dictionary import Dictionary
from gensim.corpora.mmcorpus import MmCorpus
from gensim.test.utils import datapath
import csv
import time
import statistics
import warnings
warnings.filterwarnings('ignore')  # To ignore all warnings that arise here to enhance clarity


# Fit LDA model and create pyLDAvis prepared object's topicInfo

In [0]:
path = '/content/drive/My Drive/sample_data/'

In [0]:
path = '/content/drive/My Drive/sample_data/'


dictionary = Dictionary.load(path + 'GH_full_processed_Dictionary.dict')
corpus = MmCorpus(datapath(path + 'corpus_processed_GH_full.mm'))

texts = []
with open(path + 'GH_full_processed_corpus.csv', 'r') as f:
    reader = csv.reader(f)
    texts = list(reader)

terms = []
for (key, value) in dictionary.iteritems():
  terms.append(value)

# Sampling using Freq Distr

In [0]:
def buildFreqDistr(corpus):
  vocab_tf={}
  for i in corpus:
      for item,count in dict(i).items():
          if item in vocab_tf:
              vocab_tf[item]+=count
          else:
              vocab_tf[item] = count

  freqCounts = {}
  for key, value in vocab_tf.items():
    freqCounts[dictionary[key]] = value

  total_tokens = 0
  for i in freqCounts.values():
    total_tokens += i

  freq_distr = {}
  for key, value in freqCounts.items():
    freq_distr[key] = value / total_tokens
  return freq_distr

def generateRandomSamples(freq_distr, size):
  return list(np.random.choice(list(freq_distr.keys()), size, p=list(freq_distr.values())))

In [7]:
k = [50]
lda_models = {}
term_emb = {}
#preparedObj = {}

for topic_num in k:
  print(topic_num)
  model = LdaTransformer(id2word=dictionary, num_topics=topic_num, alpha='auto', eta=0.001, iterations=400, random_state=2019)
  lda = model.fit(corpus)
  lda = lda.gensim_model
  term_topic_matrix = lda.get_topics()

  lda_models[topic_num] = lda
  term_emb[topic_num] = pd.DataFrame(term_topic_matrix, columns=terms)

  #t0 = time.time()
  #topic_info = get_topic_info(lda, corpus, dictionary)
  #preparedObj[topic_num] = topic_info
  #print(time.time() - t0, " seconds for an iteration")

50


In [0]:
import shelve

def load_data(path, num_topics):
  with shelve.open(path) as db:
    my_dict = db[str(num_topics)]
  return my_dict

def store_data(path, k):
  with shelve.open(path) as db:  
    for num_topics in k:
      db[str(num_topics)] = preparedObj[num_topics]

#store_data("/content/drive/My Drive/Expertise Experiments/topicInfo.shlf", k = [17, 14, 21, 28, 19, 11])

In [0]:
k = [17, 14, 21, 28, 19, 11]
preparedObj = {}
for i in k:
  preparedObj[i] = load_data("/content/drive/My Drive/Expertise Experiments/topicInfo.shlf", i)

# Load in human annotations

In [0]:
def intersect(a, b):
    """ return the intersection of two lists """
    return list(set(a) & set(b))

def union(a, b):
    """ return the union of two lists """
    return list(set(a) | set(b))

def load_annotations():
  SO_annotation = pd.read_csv('/content/drive/My Drive/Expertise Experiments/SO_annotations_processed.csv', header = 0,
                        names = ["sample_ID","profile_url","unified_Id","internal_ID","Annotator_1","Annotator_2", "Processed_Annotator_1", "Processed_Annotator_2"])

  GH_annotation = pd.read_csv('/content/drive/My Drive/Expertise Experiments/GH_annotations_processed.csv', header = 0, 
                        names = ["sample_ID","profile_url","unified_Id","internal_ID","Annotator_1","Annotator_2", "Processed_Annotator_1", "Processed_Annotator_2"])
  
  GH_IDs = GH_annotation["internal_ID"]
  SO_IDs = SO_annotation["internal_ID"]

  GH_annotation_intersect = {}
  GH_annotation_union = {}
  SO_annotation_intersect = {}
  SO_annotation_union = {}

  for index, row in SO_annotation.iterrows():
    a1 = row['Processed_Annotator_1'].split(";")
    a2 = row['Processed_Annotator_2'].split(";")

    if '' in a1:
      a1.remove('')
    if '' in a2:
      a2.remove('')

    SO_annotation_intersect[row['internal_ID']] = intersect(a1, a2)
    SO_annotation_union[row['internal_ID']] = union(a1, a2)

  for index, row in GH_annotation.iterrows():
    a1 = row['Processed_Annotator_1'].split(";")
    a2 = row['Processed_Annotator_2'].split(";")

    if '' in a1:
      a1.remove('')
    if '' in a2:
      a2.remove('')

    GH_annotation_intersect[row['internal_ID']] = intersect(a1, a2)
    GH_annotation_union[row['internal_ID']] = union(a1, a2)

  return GH_IDs, SO_IDs, GH_annotation_intersect, GH_annotation_union, SO_annotation_intersect, SO_annotation_union

In [0]:
GH_IDs, SO_IDs, GH_annotation_intersect, GH_annotation_union, SO_annotation_intersect, SO_annotation_union = load_annotations()

In [20]:
counter = 1
for ann in SO_annotation_union.values():
  print(counter, "&", ann, ' \\\\ ')
  counter+=1

1 & ['aspdot_net', 'ui', 'visual_studio', 'ios', 'localization', 'audiotoolbox', 'debugging', 'json', 'oriented', 'jpa', 'pcl', 'swift', 'svg', 'cisco', 'ruby', 'angularjs', 'mvc', 'object', 'xamarin', 'xml', 'android', 'sql', 'javascript', 'ajax', 'bootstrap', 'jquery', 'spring', 'router', 'sqlite', 'identicon', 'ubuntu', 'java', 'html']  \\ 
2 & ['routing', 'server', 'linux', 'cron', 'json', 'cryptography', 'based', 'dvcs', 'packages', 'event', 'macros', 'cryptographically', 'python', 'random', 'source', 'openvpn', 'sql', 'javascript', 'desktop', 'latex', 'macos', 'hash', 'drive', 'browser', 'open', 'mercurial', 'remote', 'debugging', 'audio', 'tex', 'firefox', 'inheritance', 'rebooting', 'enough', 'hard', 'ajax', 'unix', 'files', 'raspbian', 'rxjs', 'networking', 'ubuntu', 'windows', 'vcs', 'git']  \\ 
3 & ['scala', 'interpreters', 'classes', 'c', 'polymorphic', 'monads', 'data', 'python', 'random', 'parsing', 'datatypes', 'concurrency', 'type', 'replicate', 'generate', 'pattern', '

In [0]:
union_length = []
for k, v in GH_annotation_union.items():
  union_length.append(len(v))

intersect_length = []
for k, v in GH_annotation_intersect.items():
  intersect_length.append(len(v))

intersect_np = np.array(intersect_length)
print("intersect_avg = ", np.average(intersect_np))

union_np = np.array(union_length)
print("union_avg = ", np.average(union_np))

intersect_avg =  6.16
union_avg =  33.72


# Create user and topic embeddings using SO_Word2Vec_200

In [0]:
word_vectors = KeyedVectors.load("/content/drive/My Drive/embeddings/SO_pre-trained_vectors.kv", mmap='r')

CUSTOM_FILTERS = [lambda x: strip_numeric, remove_stopwords]

def word2vec_embedding_lookup(words):
  vectors = []
  for w in words:
    try:
      vec = word_vectors[w]
      vectors.append(vec)
    except:
      try:
        w_transformed = w.replace(".", "").replace("=", "").replace("-", "").replace("*", "").replace("'", "").replace("`", "").replace("|", "").replace('\\', "").replace("/", "").replace("$", "").replace("^", "").replace("#", "").replace("&", "").replace("@", "")
        vec = word_vectors[w_transformed]
        vectors.append(vec)
      except:
        try:
          w_stripped = preprocess_string(w_transformed, CUSTOM_FILTERS)
          vec = word_vectors[w_stripped]
          vectors.append(vec)
        except:
          continue
  return np.array(vectors)

In [0]:
def get_user_emb():
  user_embeddings = []
  for i in range(0,len(texts)):
    word_vectors = word2vec_embedding_lookup(list(set(texts[i]).intersection(terms)))
    try:
      feature_vector = np.max(word_vectors, axis=0)
      user_embeddings.append(feature_vector)
    except ValueError:
      user_embeddings.append(np.zeros((200,)))  # 200 x 1 vector of 0's, since the word2vec model is 200 dimensional
  return np.array(user_embeddings)

def get_topic_emb(lda):
  avg_topic_emb = []
  max_topic_emb = []
  number_of_topicWords = 20
  topic_num = lda.num_topics 

  for topic in range(0, topic_num):  # for each topic inside a specific model
    results = lda.show_topic(topic, topn=number_of_topicWords)

    topic_words = []
    for i in range(0,number_of_topicWords): # for each topic word inside a topic
      topic_words.append(results[i][0])

    word_vectors = word2vec_embedding_lookup(topic_words)
    avg_feature_vector = np.average(word_vectors, axis=0)
    max_feature_vector = np.max(word_vectors, axis=0)

    avg_topic_emb.append(avg_feature_vector)
    max_topic_emb.append(max_feature_vector)
  return np.asarray(avg_topic_emb), np.asarray(max_topic_emb)

# Create topic info data frame by calling get_topic_info(lda_model, corpus, dictionary)

In [0]:
from __future__ import absolute_import
import funcy as fp
import numpy as np
from scipy.sparse import issparse

def get_topic_info(topic_model, corpus, dictionary, doc_topic_dist=None):
  opts = fp.merge(pyLDAvis_prepare(topic_model, corpus, dictionary, doc_topic_dist))
  return my_prepare(**opts)
  
def _chunks(l, n):
    """ Yield successive n-sized chunks from l.
    """
    for i in range(0, len(l), n):
        yield l[i:i + n]


def _job_chunks(l, n_jobs):
    n_chunks = n_jobs
    if n_jobs < 0:
        # so, have n chunks if we are using all n cores/cpus
        n_chunks = cpu_count() + 1 - n_jobs

    return _chunks(l, n_chunks)


def _find_relevance(log_ttd, log_lift, R, lambda_):
    relevance = lambda_ * log_ttd + (1 - lambda_) * log_lift
    return relevance.T.apply(lambda s: s.sort_values(ascending=False).index).head(R)


def _find_relevance_chunks(log_ttd, log_lift, R, lambda_seq):
    return pd.concat([_find_relevance(log_ttd, log_lift, R, l) for l in lambda_seq])



def _df_with_names(data, index_name, columns_name):
    if type(data) == pd.DataFrame:
        # we want our index to be numbered
        df = pd.DataFrame(data.values)
    else:
        df = pd.DataFrame(data)
    df.index.name = index_name
    df.columns.name = columns_name
    return df


def _series_with_name(data, name):
    if type(data) == pd.Series:
        data.name = name
        # ensures a numeric index
        return data.reset_index()[name]
    else:
        return pd.Series(data, name=name)


def _topic_info(topic_term_dists, topic_proportion, term_frequency, term_topic_freq,
                vocab, lambda_step, R, n_jobs):
    # marginal distribution over terms (width of blue bars)
    term_proportion = term_frequency / term_frequency.sum()

    # compute the distinctiveness and saliency of the terms:
    # this determines the R terms that are displayed when no topic is selected
    topic_given_term = topic_term_dists / topic_term_dists.sum()
    kernel = (topic_given_term * np.log((topic_given_term.T / topic_proportion).T))
    distinctiveness = kernel.sum()
    saliency = term_proportion * distinctiveness
    # Order the terms for the "default" view by decreasing saliency:
    default_term_info = pd.DataFrame({
        'saliency': saliency,
        'Term': vocab,
        'Freq': term_frequency,
        'Total': term_frequency,
        'Category': 'Default'})
    default_term_info = default_term_info.sort_values(
        by='saliency', ascending=False).head(R).drop('saliency', 1)
    # Rounding Freq and Total to integer values to match LDAvis code:
    default_term_info['Freq'] = np.floor(default_term_info['Freq'])
    default_term_info['Total'] = np.floor(default_term_info['Total'])
    ranks = np.arange(R, 0, -1)
    default_term_info['logprob'] = default_term_info['loglift'] = ranks

    # compute relevance and top terms for each topic
    log_lift = np.log(topic_term_dists / term_proportion)
    log_ttd = np.log(topic_term_dists)
    lambda_seq = np.arange(0, 1 + lambda_step, lambda_step)

    def topic_top_term_df(tup):
        new_topic_id, (original_topic_id, topic_terms) = tup
        term_ix = topic_terms.unique()
        return pd.DataFrame({'Term': vocab[term_ix],
                             'Freq': term_topic_freq.loc[original_topic_id, term_ix],
                             'Total': term_frequency[term_ix],
                             'logprob': log_ttd.loc[original_topic_id, term_ix].round(4),
                             'loglift': log_lift.loc[original_topic_id, term_ix].round(4),
                             'Category': 'Topic%d' % new_topic_id})

    top_terms = pd.concat(Parallel(n_jobs=n_jobs)
                          (delayed(_find_relevance_chunks)(log_ttd, log_lift, R, ls)
                          for ls in _job_chunks(lambda_seq, n_jobs)))
    topic_dfs = map(topic_top_term_df, enumerate(top_terms.T.iterrows(), 1))
    return pd.concat([default_term_info] + list(topic_dfs), sort=True)

def pyLDAvis_prepare(topic_model, corpus, dictionary, doc_topic_dists=None):
    import gensim
    if not gensim.matutils.ismatrix(corpus):
        corpus_csc = gensim.matutils.corpus2csc(corpus, num_terms=len(dictionary))
    else:
        corpus_csc = corpus
        # Need corpus to be a streaming gensim list corpus for len and inference functions below:
        corpus = gensim.matutils.Sparse2Corpus(corpus_csc)

    vocab = list(dictionary.token2id.keys())
    # TODO: add the hyperparam to smooth it out? no beta in online LDA impl.. hmm..
    # for now, I'll just make sure we don't ever get zeros...
    beta = 0.01
    fnames_argsort = np.asarray(list(dictionary.token2id.values()), dtype=np.int_)
    term_freqs = corpus_csc.sum(axis=1).A.ravel()[fnames_argsort]
    term_freqs[term_freqs == 0] = beta
    doc_lengths = corpus_csc.sum(axis=0).A.ravel()

    if hasattr(topic_model, 'lda_alpha'):
        num_topics = len(topic_model.lda_alpha)
    else:
        num_topics = topic_model.num_topics

    if doc_topic_dists is None:
        # If its an HDP model.
        if hasattr(topic_model, 'lda_beta'):
            gamma = topic_model.inference(corpus)
        else:
            gamma, _ = topic_model.inference(corpus)
        doc_topic_dists = gamma / gamma.sum(axis=1)[:, None]
    else:
        if isinstance(doc_topic_dists, list):
            #doc_topic_dists = np.matrix(gensim.matutils.corpus2dense(doc_topic_dists, num_topics).T)
            doc_topic_dists = gensim.matutils.corpus2dense(doc_topic_dists, num_topics).T
        elif issparse(doc_topic_dists):
            doc_topic_dists = doc_topic_dists.T.todense()
        doc_topic_dists = doc_topic_dists / doc_topic_dists.sum(axis=1)

    # get the topic-term distribution straight from gensim without
    # iterating over tuples
    if hasattr(topic_model, 'lda_beta'):
        topic = topic_model.lda_beta
    else:
        topic = topic_model.state.get_lambda()
    topic = topic / topic.sum(axis=1)[:, None]
    topic_term_dists = topic[:, fnames_argsort]

    topic_freq = (doc_topic_dists.T * doc_lengths).T.sum()
    topic_proportion = (topic_freq / topic_freq.sum())

    term_topic_freq = (topic_term_dists.T * topic_freq).T

    return {'topic_term_dists': topic_term_dists, 'doc_topic_dists': doc_topic_dists, 
            'doc_lengths': doc_lengths, 'vocab': vocab, 'term_frequency': term_freqs}

def my_prepare(topic_term_dists, doc_topic_dists, doc_lengths, vocab, term_frequency,
            R=30, lambda_step=0.01, n_jobs=-1, sort_topics=True):

    topic_term_dists = _df_with_names(topic_term_dists, 'topic', 'term')
    doc_topic_dists = _df_with_names(doc_topic_dists, 'doc', 'topic')
    term_frequency = _series_with_name(term_frequency, 'term_frequency')
    doc_lengths = _series_with_name(doc_lengths, 'doc_length')
    vocab = _series_with_name(vocab, 'vocab')

    topic_freq = (doc_topic_dists.T * doc_lengths).T.sum()
    if (sort_topics):
        topic_proportion = (topic_freq / topic_freq.sum()).sort_values(ascending=False)
    else:
        topic_proportion = (topic_freq / topic_freq.sum())

    topic_order = topic_proportion.index
    # reorder all data based on new ordering of topics
    topic_freq = topic_freq[topic_order]
    topic_term_dists = topic_term_dists.iloc[topic_order]
    doc_topic_dists = doc_topic_dists[topic_order]

    # token counts for each term-topic combination (widths of red bars)
    term_topic_freq = (topic_term_dists.T * topic_freq).T
    term_frequency = np.sum(term_topic_freq, axis=0)

    topic_info = _topic_info(topic_term_dists, topic_proportion,
                             term_frequency, term_topic_freq, vocab, lambda_step, R, n_jobs)
    return topic_info

# Define techniques

## Word2Vec user and topic Embeddings using Max and Avg pooling

In [0]:
def run_Word2Vec_emb(lda, threshold, maxPool):
  user_vectors = get_user_emb()
  avg_topic_vectors, max_topic_vectors = get_topic_emb(lda)
  
  if maxPool:
    topic_vectors = max_topic_vectors
  else:
    topic_vectors = avg_topic_vectors
  
  cos_sim = cosine_similarity(user_vectors, topic_vectors)
  #np.savetxt("/content/drive/My Drive/Expertise Experiments/SO_Full_word2vec_k=17_Manual_CosSim_user_vectors_topic_vectors.csv", cos_sim, delimiter=",")
  user_topic_mapping = create_user_topic_mapping(cos_sim, threshold)
  return user_topic_mapping

## LDA_topicEmbedding using Max-pooling and Avg-pooling



In [0]:
def get_LDA_user_emb(topic_num):
  user_embeddings = []
  for i in range(0,len(texts)):
    word_vectors = embedding_lookup(list(set(texts[i]).intersection(terms)), topic_num)
    try:
      feature_vector = np.max(word_vectors, axis=1)
      user_embeddings.append(feature_vector)
    except ValueError:
      user_embeddings.append(np.zeros((topic_num,)))
  return np.array(user_embeddings)

def get_LDA_topic_emb(topic_num):
  avg_topic_emb = []
  max_topic_emb = []
  number_of_topicWords = 20

  for topic in range(0, topic_num):  # for each topic inside a specific model
    lda = lda_models[topic_num]
    results = lda.show_topic(topic, topn=number_of_topicWords)

    topic_words = []
    for i in range(0,number_of_topicWords):
      topic_words.append(results[i][0])

    word_vectors = embedding_lookup(topic_words, topic_num)
    avg_feature_vector = np.average(word_vectors, axis=1)
    max_feature_vector = np.max(word_vectors, axis=1)

    avg_topic_emb.append(avg_feature_vector)
    max_topic_emb.append(max_feature_vector)

  avg_topic_vectors = np.array(avg_topic_emb)
  max_topic_vectors = np.array(max_topic_emb)
  return avg_topic_vectors, max_topic_vectors

In [0]:
def jaccard_sim(A, B):
  return 1 - jaccard_distance(set(A), set(B))     # jaccard sim = 1 - jaccard distance

In [0]:
def run_LDA_emb(topic_num, threshold, maxPool = True):
  user_vectors = get_LDA_user_emb(topic_num)
  avg_topic_vectors, max_topic_vectors = get_LDA_topic_emb(topic_num)

  if maxPool:
    topic_vectors = max_topic_vectors
  else:
    topic_vectors = avg_topic_vectors

  cos_sim = cosine_similarity(user_vectors, topic_vectors)
  #np.savetxt("/content/drive/My Drive/Expertise Experiments/LDA_k=17_cos_sim_user_vectors_topic_vectors.csv", cos_sim, delimiter=",")
  user_topic_mapping = create_user_topic_mapping(cos_sim, threshold)
  return user_topic_mapping

In [0]:
def create_user_topic_mapping(cos_sims, threshold):
  user_topic_mapping = {}
  for user_i in range(0, 83550):   # counting for users 0 --> 83549
    try:
      user_topic_mapping[user_i] = ['Topic' + str(index+1) for index, value in enumerate(cos_sims[user_i]) if value > threshold]
    except:
      user_topic_mapping[user_i] = []
  return user_topic_mapping

In [0]:
def get_user_expertise(topicInfo, user_i, topic_terms):
  optimal_lambda_val = 0.8
  expertise = get_relevant_terms(topicInfo, user_i, optimal_lambda_val, topic_terms)
  return expertise

In [0]:
def get_relevant_terms(topic_info, topics, _lambda, term_num):
  """Retuns a list of top-n keywords (where n = term_num) that have the highest relevance score for the topics the the user is in."""

  tdf = pd.DataFrame(topic_info[topic_info.Category.isin(topics)])
  stdf = tdf.assign(relevance=_lambda * tdf['logprob'] + (1 - _lambda) * tdf['loglift'])
  new_df = stdf.sort_values('relevance', ascending=False)

  term_list = new_df['Term'].tolist()
  if '-PRON-' in term_list:
    term_list.remove('-PRON-')
  if ' ' in term_list:
    term_list.remove(' ')

  if term_num > len(term_list):
    return term_list
  else:
    return term_list[0:term_num]

## LDA Topic Distribution based Expertise

In [0]:
def LDA_topicDistr(lda_model, topicInfo, user_i, threshold, topic_terms):
  user_topic_membership = create_user_topic_thresholding(lda_model, user_i, threshold)
  user_expertise = get_expertise_for_user_i(topicInfo, user_topic_membership, topic_terms)
  return user_expertise

In [0]:
def create_user_topic_thresholding(lda_model, user_i, threshold):
  user_i_topic_distr = lda_model.get_document_topics(bow = corpus[user_i], minimum_probability = threshold)

  topic_memberships = []
  for topic in user_i_topic_distr:
    topic_memberships.append('Topic' + str(topic[0]+1))   # topics are 0 to k-1, so offset by 1, since pyLDAvis indexes from 1 to k

  return topic_memberships

In [0]:
def get_expertise_for_user_i(topicInfo, user_topic_membership, topic_terms):
  optimal_lambda_val = 0.8
  expertise = get_relevant_terms(topicInfo, user_topic_membership, optimal_lambda_val, topic_terms)
  return expertise

# Create Evaluation functions

In [0]:
def getExistingWordsFromModel(words):
  """ Checks if a list of words are in the dictionary of the word2vec model """
  CUSTOM_FILTERS = [lambda x: strip_numeric, remove_stopwords]
  res = []
  for w in words:
    try:
      vec = word_vectors[w]
      res.append(w)
    except:
      try:
        w_transformed = w.replace(".", "").replace("=", "").replace("-", "").replace("*", "").replace("'", "").replace("`", "").replace("|", "").replace('\\', "").replace("/", "").replace("$", "").replace("^", "").replace("&", "").replace("@", "").replace("%", "")
        vec = word_vectors[w_transformed]
        res.append(w_transformed)
      except:
         try:
          w_stripped = preprocess_string(w_transformed, CUSTOM_FILTERS)
          vec = word_vectors[w_stripped]
          res.append(w_stripped)
         except:
           continue
  return res

In [0]:
def evaluate_random_model():
  BLEU_scores = []
  jacc_similarity = []
  cos_similarity = []

  freq_distr = buildFreqDistr(corpus)
  counter = 0

  for user_i in GH_IDs:
    if counter % 10 == 0:
      print(counter)
    counter += 1

    # or if you want intersect, use SO_annotation_intersect
    annotation = GH_annotation_union[user_i]
    smooth = SmoothingFunction()
    a = getExistingWordsFromModel(annotation)

    bleu = []
    jaccard = []
    cosine = []

    for i in range(0, 10000):
      model_hypothesis = generateRandomSamples(freq_distr, len(annotation))

      # 1-gram individual BLEU with smoothing function
      temp = sentence_bleu(references = [annotation], hypothesis = model_hypothesis, weights = (1, 0, 0, 0), smoothing_function = smooth.method1)
      bleu.append(temp)
      
      # calculate Jaccard similarity between annotation and model hypothesis 
      jaccard.append(jaccard_sim(annotation, model_hypothesis))

      # Compute cosine similarity between annotation and model hypothesis 
      b = getExistingWordsFromModel(model_hypothesis)

      if len(a) > 0 and len(b) > 0:
        cosine.append(word_vectors.n_similarity(a, b))
      
    bleu_np = np.asarray(bleu)
    jacc_np = np.asarray(jaccard)
    cos_np = np.asarray(cosine)

    BLEU_scores.append(np.mean(bleu_np))
    jacc_similarity.append(np.mean(jacc_np))
    cos_similarity.append(np.mean(cos_np))

  return BLEU_scores, jacc_similarity, cos_similarity

In [0]:
BLEU_scores, jacc_similarity, cos_similarity = evaluate_random_model()

0
10
20
30
40
50
60
70
80
90


In [0]:
print(np.average(cos_similarity))
print(np.average(jacc_similarity))
print(np.average(BLEU_scores))

0.59616256
0.02859637729286554
0.054090284075987934


In [0]:
def evaluate_LDA_topicDistr(lda_model, topicInfo, threshold_t):
  BLEU_scores = []
  jacc_similarity = []
  cos_similarity = []

  recall = []
  precision = []
  fscore = []

  for user_i in GH_IDs:
    # or if you want intersect, use SO_annotation_intersect
    annotation = GH_annotation_union[user_i]

    model_hypothesis = LDA_topicDistr(lda_model, topicInfo, user_i, threshold_t, topic_terms = len(annotation))

    # 1-gram individual BLEU with smoothing function
    smooth = SmoothingFunction()
    BLEU_score = sentence_bleu(references = [annotation], hypothesis = model_hypothesis, weights = (1, 0, 0, 0), smoothing_function = smooth.method1)
    BLEU_scores.append(BLEU_score)

    # calculate precision, recall, fscore
    if len(annotation) == len(model_hypothesis):
      micro_res = precision_recall_fscore_support(y_true = np.array(annotation), y_pred = np.array(model_hypothesis), average='micro')
      precision.append(micro_res[0])
      recall.append(micro_res[1])
      fscore.append(micro_res[2])

    # calculate Jaccard similarity between annotation and model hypothesis 
    jaccard_simm = jaccard_sim(annotation, model_hypothesis)
    jacc_similarity.append(jaccard_simm)

    # Compute cosine similarity between annotation and model hypothesis 
    a = getExistingWordsFromModel(annotation)
    b = getExistingWordsFromModel(model_hypothesis)

    if len(a) > 0 and len(b) > 0:
      cos_sim = word_vectors.n_similarity(a, b)
      print("Annotation", annotation, "\nModel Hypothesis:", model_hypothesis, "\nCos Sim = ", cos_sim, "\n")
      cos_similarity.append(cos_sim)

  return BLEU_scores, jacc_similarity, cos_similarity, precision, recall, fscore

In [0]:
def evaluate_LDA_topicEmb(topic_num, topicInfo, threshold, maxPool):
  user_topic_mapping = run_LDA_emb(topic_num, threshold, maxPool)

  BLEU_scores = []
  jacc_similarity = []
  cos_similarity = []

  recall = []
  precision = []
  fscore = []

  for user_i in GH_IDs:
    # or if you want intersect, use SO_annotation_intersect
    annotation = GH_annotation_union[user_i]

    model_hypothesis = get_user_expertise(topicInfo, user_topic_mapping[user_i], topic_terms = len(annotation))

    # 1-gram individual BLEU with smoothing function
    smooth = SmoothingFunction()
    BLEU_score = sentence_bleu(references = [annotation], hypothesis = model_hypothesis, 
                               weights = (1, 0, 0, 0), smoothing_function = smooth.method1)
    BLEU_scores.append(BLEU_score)

    # calculate precision, recall, fscore
    if len(annotation) == len(model_hypothesis):
      micro_res = precision_recall_fscore_support(y_true = np.array(annotation), y_pred = np.array(model_hypothesis), average='micro')
      precision.append(micro_res[0])
      recall.append(micro_res[1])
      fscore.append(micro_res[2])

    # calculate Jaccard similarity between annotation and model hypothesis 
    jaccard_simm = jaccard_sim(annotation, model_hypothesis)
    jacc_similarity.append(jaccard_simm)

    # Compute cosine similarity between annotation and model hypothesis 
    a = getExistingWordsFromModel(annotation)
    b = getExistingWordsFromModel(model_hypothesis)
    
    if len(a) > 0 and len(b) > 0:
      cos_sim = word_vectors.n_similarity(a, b)
      cos_similarity.append(cos_sim)

  return BLEU_scores, jacc_similarity, cos_similarity, precision, recall, fscore

In [0]:
def evaluate_Word2Vec_Emb(lda, topicInfo, threshold, maxPool):
  user_topic_mapping = run_Word2Vec_emb(lda, threshold, maxPool)

  BLEU_scores = []
  jacc_similarity = []
  cos_similarity = []

  recall = []
  precision = []
  fscore = []

  for user_i in GH_IDs:
    # or if you want intersect, use SO_annotation_intersect
    annotation = GH_annotation_union[user_i]
    model_hypothesis = get_user_expertise(topicInfo, user_topic_mapping[user_i], topic_terms = len(annotation))
    print("\n\tannotation: ", annotation, " \n\tmodel_hypothesis: ", model_hypothesis, "\n")

    # 1-gram individual BLEU with smoothing function
    smooth = SmoothingFunction()
    BLEU_score = sentence_bleu(references = [annotation], hypothesis = model_hypothesis, 
                               weights = (1, 0, 0, 0), smoothing_function = smooth.method1)
    BLEU_scores.append(BLEU_score)

    # calculate precision, recall, fscore
    if len(annotation) == len(model_hypothesis):
      micro_res = precision_recall_fscore_support(y_true = np.array(annotation), y_pred = np.array(model_hypothesis), average='micro')
      precision.append(micro_res[0])
      recall.append(micro_res[1])
      fscore.append(micro_res[2])

    # calculate Jaccard similarity between annotation and model hypothesis 
    jaccard_simm = jaccard_sim(annotation, model_hypothesis)
    jacc_similarity.append(jaccard_simm)

    # Compute cosine similarity between annotation and model hypothesis 
    a = getExistingWordsFromModel(annotation)
    b = getExistingWordsFromModel(model_hypothesis)
    
    if len(a) > 0 and len(b) > 0:
      cos_sim = word_vectors.n_similarity(a, b)
      cos_similarity.append(cos_sim)

  return BLEU_scores, jacc_similarity, cos_similarity, precision, recall, fscore

Run experiments using a random model, based on sampling from the frequency distribution

In [48]:
"""
## Run Experiments for LDA_topic distribution based expertise on SO data, using SO_past model
"""

def main_topicDistr(lda_model, topic_info):
  #threshold = [0.10, 0.12, 0.14, 0.16, 0.18, 0.20,
  #           0.22, 0.24, 0.26, 0.28, 0.30,
  #           0.32, 0.34, 0.36, 0.38, 0.40,
  #           0.42, 0.44, 0.46, 0.48, 0.50]
  threshold = [0.30]           
  mean_jaccard = []
  mean_bleu = []
  mean_cos = []
  mean_fscore = []

  for t in threshold:
    BLEU_scores, jacc_sim, cos_sim, precision, recall, fscore = evaluate_LDA_topicDistr(lda_model, topic_info, t)
    bleu_np = np.asarray(BLEU_scores)
    jacc_np = np.asarray(jacc_sim)
    cos_np = np.asarray(cos_sim)
    fscore_np = np.asarray(fscore)

    print("Threshold = ",t, ".. Cos Sim = ", np.mean(cos_np))

    mean_jaccard.append( np.mean(jacc_np) )
    mean_bleu.append( np.mean(bleu_np) )
    mean_cos.append( np.mean(cos_np) )
    mean_fscore.append( np.mean(fscore_np) )
  return np.max( np.asarray(mean_bleu) ), np.max( np.asarray(mean_jaccard) ), np.max( np.asarray(mean_cos) ), np.max( np.asarray(mean_fscore) )

"""##Run Experiments for Word2vec user and topic Embedding using Avg-pooling on SO data"""


'##Run Experiments for Word2vec user and topic Embedding using Avg-pooling on SO data'

In [0]:

"""## Run Experiments for LDA_topicEmbedding using Avg-pooling on SO data, using SO_full model"""

def main_LDA_avgEmb(lda_model, topic_info, term_emb):
  threshold_values = [0.40, 0.42, 0.44, 0.46, 0.48, 0.50, 0.52, 0.54, 0.56, 
                    0.58, 0.60, 0.62, 0.64, 0.66, 0.68, 0.70, 0.72, 0.74, 
                    0.76, 0.78, 0.80, 0.82, 0.84, 0.86, 0.88, 0.90]
  mean_jaccard = []
  mean_bleu = []
  mean_cos = []
  mean_fscore = []
  
  for threshold in threshold_values:
    BLEU_scores, jacc_sim, cos_sim, precision, recall, fscore = evaluate_LDA_topicEmb(lda_model, topic_info, term_emb, threshold, maxPool=False)
    bleu_np = np.asarray(BLEU_scores)
    jacc_np = np.asarray(jacc_sim)
    cos_np = np.asarray(cos_sim)
    fscore_np = np.asarray(fscore)

    mean_jaccard.append( np.mean(jacc_np) )
    mean_bleu.append( np.mean(bleu_np) )
    mean_cos.append( np.mean(cos_np) )
    mean_fscore.append( np.mean(fscore_np) )
  return np.max( np.asarray(mean_bleu) ), np.max( np.asarray(mean_jaccard) ), np.max( np.asarray(mean_cos) ), np.max( np.asarray(mean_fscore) )

"""## Run Experiments for LDA_topicEmbedding using Max-pooling on SO data, using SO_past model"""

def main_LDA_maxEmb(lda_model, topic_info, term_emb):
  threshold_values = [0.20, 0.25, 0.30, 0.35, 0.40, 0.42, 0.44, 0.46, 0.48, 
                      0.50, 0.52, 0.54, 0.56, 0.58, 0.60, 0.62, 0.64, 0.66,
                      0.68, 0.70, 0.72, 0.74, 0.76, 0.78, 0.80, 0.82, 0.84, 0.86, 0.88, 0.90] 
  mean_jaccard = []
  mean_bleu = []
  mean_cos = []
  mean_fscore = []

  for threshold in threshold_values:
    BLEU_scores, jacc_sim, cos_sim, precision, recall, fscore = evaluate_LDA_topicEmb(lda_model, topic_info, term_emb, threshold, maxPool=True)
    bleu_np = np.asarray(BLEU_scores)
    jacc_np = np.asarray(jacc_sim)
    cos_np = np.asarray(cos_sim)
    fscore_np = np.asarray(fscore)

    mean_jaccard.append( np.mean(jacc_np) )
    mean_bleu.append( np.mean(bleu_np) )
    mean_cos.append( np.mean(cos_np) )
    mean_fscore.append( np.mean(fscore_np) )
  return np.max( np.asarray(mean_bleu) ), np.max( np.asarray(mean_jaccard) ), np.max( np.asarray(mean_cos) ), np.max( np.asarray(mean_fscore) )


In [0]:
def main_Word2Vec_AvgEmb(lda_model, topic_info):
  threshold_values = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.10,
                    0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.20]
  mean_jaccard = []
  mean_bleu = []
  mean_cos = []
  mean_fscore = []

  for threshold in threshold_values:
    BLEU_scores, jacc_sim, cos_sim, precision, recall, fscore = evaluate_Word2Vec_Emb(lda=lda_model, topicInfo=topic_info, threshold=threshold, maxPool=False)
    bleu_np = np.asarray(BLEU_scores)
    jacc_np = np.asarray(jacc_sim)
    cos_np = np.asarray(cos_sim)
    fscore_np = np.asarray(fscore)

    mean_jaccard.append( np.mean(jacc_np) )
    mean_bleu.append( np.mean(bleu_np) )
    mean_cos.append( np.mean(cos_np) )
    mean_fscore.append( np.mean(fscore_np) )
    print("threshold = ", threshold, "Avg_Cosine = ", mean_cos)
  return np.max( np.asarray(mean_bleu) ), np.max( np.asarray(mean_jaccard) ), np.max( np.asarray(mean_cos) ), np.max( np.asarray(mean_fscore) )

"""## Run Experiments for Word2vec user and topic Embedding using Max-pooling on SO data"""

def main_Word2Vec_MaxEmb(lda_model, topic_info):
  threshold_values = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.10,
                    0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.20,
                    0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.30,
                    0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.40]
  mean_jaccard = []
  mean_bleu = []
  mean_cos = []
  mean_fscore = []

  for threshold in threshold_values:
    BLEU_scores, jacc_sim, cos_sim, precision, recall, fscore = evaluate_Word2Vec_Emb(lda_model, topicInfo=topic_info, threshold = threshold, maxPool=True)
    bleu_np = np.asarray(BLEU_scores)
    jacc_np = np.asarray(jacc_sim)
    cos_np = np.asarray(cos_sim)
    fscore_np = np.asarray(fscore)

    mean_jaccard.append( np.mean(jacc_np) )
    mean_bleu.append( np.mean(bleu_np) )
    mean_cos.append( np.mean(cos_np) )
    mean_fscore.append( np.mean(fscore_np) )
    print("threshold = ", threshold, "Avg_Cosine = ", mean_cos)
  return np.max( np.asarray(mean_bleu) ), np.max( np.asarray(mean_jaccard) ), np.max( np.asarray(mean_cos) ), np.max( np.asarray(mean_fscore) )

# Main

In [0]:
def evaluateModel(lda_model, topic_info, term_emb, mode):
  if mode == 1:
    max_bleu, max_jaccard, max_cos, max_fscore = main_topicDistr(lda_model, topic_info)
  elif mode == 2:
    max_bleu, max_jaccard, max_cos, max_fscore = main_LDA_avgEmb(lda_model, topic_info, term_emb)
  elif mode == 3:
    max_bleu, max_jaccard, max_cos, max_fscore = main_LDA_maxEmb(lda_model, topic_info, term_emb)
  elif mode == 4:
    max_bleu, max_jaccard, max_cos, max_fscore = main_Word2Vec_AvgEmb(lda_model, topic_info)
  elif mode == 5:
    max_bleu, max_jaccard, max_cos, max_fscore = main_Word2Vec_MaxEmb(lda_model, topic_info)
  print(max_bleu, max_jaccard, max_cos, max_fscore)


topic_info = get_topic_info(lda_models[50], corpus, dictionary)

In [49]:
evaluateModel(lda_models[50], topic_info, term_emb[topic_num], mode = 1)

Annotation ['nix_os', 'scala', 'oriented', 'c', 'programming', 'sbt', 'dhall', 'python', 'go', 'apache', 'shell', 'sql', 'homebrew', 'shell_programming', 'kubernetes', 'haskell', 'mac_os', 'java', 'html', 'css', 'wireshark', 'functional_programming', 'software_engineering', 'libraries', 'dhall_lang', 'hackage', 'operating_systems', 'ruby', 'c++', 'nix', 'object', 'airflow', 'postgres', 'helm', 'postres', 'morte_library'] 
Model Hypothesis: ['puppet', 'plugin', 'vagrant', 'jenkins', 'module', 'plugins', 'git', 'zsh', 'packer', 'tool', 'shell', 'agent', 'gitlab', 'managing', 'infrastructure', 'manage', 'hub', 'oracle', 'salt', 'ant', 'metrics', 'configuration.', 'rpm', 'communitydriven', 'osx', 'provisioning', 'server', 'framework', 'spice', 'configuration', 'java', 'debian', 'morning', 'ssl', 'management', 'datadog'] 
Cos Sim =  0.6097158 

Annotation ['scala', 'web_scraping', 'npm', 'programming', 'nestor', 'go', 'c#', 'shell', 'javascript', 'mongoosejavascript', 'expressjavascript', '

In [0]:
GH_IDs, SO_IDs, GH_annotation_intersect, GH_annotation_union, SO_annotation_intersect, SO_annotation_union = load_annotations()

dictionary = Dictionary.load(path + 'GH_full_processed_Dictionary.dict')
corpus = MmCorpus(datapath(path + 'corpus_processed_GH_full.mm'))

texts = []
with open(path + 'GH_full_processed_corpus.csv', 'r') as f:
    reader = csv.reader(f)
    texts = list(reader)

terms = []
for (key, value) in dictionary.iteritems():
  terms.append(value)
	
def evaluateModel(lda_model, topic_info, term_emb, mode):
  if mode == 1:
    max_bleu, max_jaccard, max_cos, max_fscore = main_topicDistr(lda_model, topic_info)
  elif mode == 2:
    max_bleu, max_jaccard, max_cos, max_fscore = main_LDA_avgEmb(lda_model, topic_info, term_emb)
  elif mode == 3:
    max_bleu, max_jaccard, max_cos, max_fscore = main_LDA_maxEmb(lda_model, topic_info, term_emb)
  elif mode == 4:
    max_bleu, max_jaccard, max_cos, max_fscore = main_Word2Vec_AvgEmb(lda_model, topic_info)
  elif mode == 5:
    max_bleu, max_jaccard, max_cos, max_fscore = main_Word2Vec_MaxEmb(lda_model, topic_info)
  print(max_bleu, max_jaccard, max_cos, max_fscore)


k = [16]
for i in range(10,101):
	k.append(i)
	
beta = [0.05]

for topic_num in k:
  print("----- Progress: k= ", topic_num ,"----")
  for b in beta:
    model = LdaTransformer(id2word=dictionary, num_topics = topic_num, alpha='auto', eta = b, iterations=100, random_state=2019)
    lda = model.fit(corpus)
    term_topic_matrix = lda.gensim_model.get_topics()
    term_emb = pd.DataFrame(term_topic_matrix, columns=terms)
    topic_info = get_topic_info(lda.gensim_model, corpus, dictionary)
    evaluateModel(lda.gensim_model, topic_info, term_emb, mode = 4)

----- Progress: k=  16 ----


In [0]:
try:
    import pyLDAvis.gensim
    CAN_VISUALIZE = True
    pyLDAvis.enable_notebook()
    from IPython.display import display
except ImportError:
    print("SKIP: please install pyLDAvis")
    CAN_VISUALIZE = False

saving_path = "/content/drive/My Drive/sample_data/"
pyLDAvis.enable_notebook()
if CAN_VISUALIZE:
    prepared = pyLDAvis.gensim.prepare(lda.gensim_model, corpus, dictionary, mds='tsne')
    pyLDAvis.save_html(prepared, saving_path + 'bestLDA_GH_full_TSNE_topic16_beta0.05.html')
    display(pyLDAvis.display(prepared))

    prepared = pyLDAvis.gensim.prepare(lda.gensim_model, corpus, dictionary)
    pyLDAvis.save_html(prepared, saving_path + 'bestLDA_GH_full_MDS_topic16_beta0.05.html')
    display(pyLDAvis.display(prepared))

    

In [0]:
#evaluateModel(lda.gensim_model, topic_info, term_emb, mode = 4)

print(evaluate_Word2Vec_Emb(lda=lda.gensim_model, topicInfo=topic_info, threshold=0.01, maxPool=False))


	annotation:  ['angularjavascript', 'java', 'nlp', 'tensorflow', 'python', 'software_engineering', 'angular', 'redux', 'javascript', 'api', 'react', 'c#', 'coding', 'html', 'unity', 'data_science', 'text', 'bert', 'programming', 'lstm', 'git', 'reactjavascript', 'ionic', 'jupyter_notebook', 'entity', 'objective_c', 'deep_learning', 'ruby', 'binary_classification', 'machine_learning', 'matching', 'entity_text_matching', 'processing']  
	model_hypothesis:  ['python', 'ruby', 'php', 'scala', 'clojure', 'javascript', 'haskell', 'c', 'rust', 'java', 'perl', 'spring', 'c++', 'rails', 'docker', 'shell', 'r', 'ansible', 'awesome', 'elm', 'react', 'django', 'elixir', 'spark', 'kotlin', 'ocaml', 'php', 'java', 'css', 'library', 'boot', 'lisp', 'plugin'] 


	annotation:  ['development', 'zsh', 'babel', 'front_end', 'rxjavascript', 'homebrew', 'software_engineering', 'python', 'travis_ci', 'shell', 'javascript', 'react', 'haskell', 'ui', 'xterm', 'coding', 'devops', 'html', 'extension', 'fedora',